This notebook combines previous disaggregation of Norway to NUTS3 level (by Max) and the raw data from the [Interannual Demand Calculator](https://github.com/martacki/demand_calculator). The work originally done by Max should be documented and stored in a "secure" location.

In [ ]:
import pandas as pd

In [ ]:
ssb_to_nuts = {
    'NO03' : 'NO081',
    'NO30' : 'NO082',
    'NO34' : 'NO020',
    'NO38' : 'NO091',
    'NO42' : 'NO092',
    'NO11' : 'NO0A1',
    'NO46' : 'NO0A2',
    'NO15' : 'NO0A3',
    'NO50' : 'NO060',
    'NO18' : 'NO071',
    'NO54' : 'NO074',
}

In [ ]:
processed_file = (
    pd
    .read_csv('/fp/projects01/ec85/models/oskar/shared_input/processed_europe_demand_1950-2021.csv',parse_dates = ['datetime'])
    .set_index('datetime')
    .rename(columns=ssb_to_nuts)
)

In [ ]:
raw_demand_file = (
    pd.read_csv('/fp/projects01/ec85/models/oskar/shared_input/raw_demand_hourly_1950_2021.csv',parse_dates=['Unnamed: 0'])
)

In [ ]:
# Double demand for all countries and add the Norwegian disaggregation.
demand_file = (
    raw_demand_file
    .rename(columns={'Unnamed: 0' : 'datetime'})
    .set_index('datetime')
    .mul(2)
    .join(processed_file[["NO020", "NO060", "NO071","NO074", "NO081", "NO082","NO091", "NO092", "NO0A1","NO0A2", "NO0A3"]])
)

In [ ]:
norway = [
    "NO020", "NO060", "NO071",
    "NO074", "NO081", "NO082",
    "NO091", "NO092", "NO0A1",
    "NO0A2", "NO0A3"
]

In [ ]:
aggregated_regions = [
    "NO020", "NO060", "NO071",
    "NO074", "NO081", "NO082",
    "NO091", "NO092", "NO0A1",
    "NO0A2", "NO0A3", "SE"
    "AT", "BE", "BG",
    "CH", "CZ", "DE",
    "DK", "EE", "ES",
    "FI", "FR", "UK",
    "GR", "HR", "HU",
    "IE", "IT", "LT",
    "LU", "LV", "NL",
    "NO", "PL","PT",
    "RO", "SE", "SI",
    "SK",
]

In [ ]:
demand_file.query("datetime > '2009-12-31 23:00:00'").query("datetime < '2011-01-01 00:00:00'").SE

In [ ]:
# Sweden 298 TWh
print(demand_file.query("datetime > '2009-12-31 23:00:00'").query("datetime < '2011-01-01 00:00:00'").SE.sum())
# Norway 220 TWh
print(demand_file.query("datetime > '2009-12-31 23:00:00'").query("datetime < '2011-01-01 00:00:00'").NO.sum())
# Finland 131 TWh
print(demand_file.query("datetime > '2009-12-31 23:00:00'").query("datetime < '2011-01-01 00:00:00'").FI.sum())

In [ ]:
year_list = [1995,2010]

for year in year_list:
    start_string = str(year - 1) + '-12-31 23:00:00'
    print(start_string)
    end_string = str(year + 1) + '-01-01 00:00:00'
    print(end_string)

    finland = demand_file.FI.to_frame().query("datetime > @start_string").query("datetime < @end_string")
    finland = finland.assign(new_demand = lambda x : (x.FI/x.FI.sum()).mul(131e6)).loc[:,'new_demand']
    demand_file.loc[start_string : end_string,'FI'] = finland
    
    sweden = demand_file.SE.to_frame().query("datetime > @start_string").query("datetime < @end_string")
    sweden = sweden.assign(new_demand = lambda x : (x.SE/x.SE.sum()).mul(298e6)).loc[:,'new_demand']
    demand_file.loc[start_string : end_string,'SE'] = sweden


In [ ]:
# Sweden 298 TWh
print(demand_file.query("datetime > '2009-12-31 23:00:00'").query("datetime < '2011-01-01 00:00:00'").SE.sum())
# Norway 220 TWh
print(demand_file.query("datetime > '2009-12-31 23:00:00'").query("datetime < '2011-01-01 00:00:00'").NO.sum())
# Finland 131 TWh
print(demand_file.query("datetime > '2009-12-31 23:00:00'").query("datetime < '2011-01-01 00:00:00'").FI.sum())

In [ ]:

year_list = [1995,2010]

for year in year_list:
    sum = 0
    start_string = str(year - 1) + '-12-31 23:00:00'
    print(start_string)
    end_string = str(year + 1) + '-01-01 00:00:00'
    print(end_string)
    for county in norway:
        share = (
            processed_file
            .query("datetime > @start_string")
            .query("datetime < @end_string")
            .sum()
            .to_frame()
            .rename(columns={0 : 'A'})
            .assign(share = lambda x : x.A/x.A.sum())
            .loc[county,['share']]
            .iloc[0]
        )
    
        temp_file = demand_file.loc[start_string : end_string,county].to_frame()
        temp_file = temp_file.assign(new_demand = lambda x : (x[county]/x[county].sum()).mul(220e6).mul(share)).loc[:,'new_demand']
        demand_file.loc[start_string : end_string,county] = temp_file
        sum += temp_file.sum()
    print(sum)

In [ ]:
norway

In [ ]:
# Norway 220 TWh

(
    demand_file
    .query("datetime > '2009-12-31 23:00:00'")
    .query("datetime < '2011-01-01 00:00:00'")
    .loc[:,['NO020','NO060','NO071','NO074','NO081','NO082','NO091','NO092','NO0A1','NO0A2','NO0A3']]
    .sum().sum()
)

In [ ]:
# Sweden 298 TWh
print(demand_file.query("datetime > '2009-12-31 23:00:00'").query("datetime < '2011-01-01 00:00:00'").SE.sum())
# Finland 131 TWh
print(demand_file.query("datetime > '2009-12-31 23:00:00'").query("datetime < '2011-01-01 00:00:00'").FI.sum())

In [ ]:
demand_file = demand_file.rename(columns={'GB' : 'UK'})

In [ ]:
demand_file.to_csv('../../resources/europe_demand_1950-2021_NEO.csv')